# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/allergy_Summaries.pkl.bz2'
Abstracts_file = 'data/allergy_Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [3]:
Summaries[24037376]

paper(title='Type 2 innate lymphoid cells control eosinophil homeostasis.', authors=['Nussbaum JC', 'Van Dyken SJ', 'von Moltke J', 'Cheng LE', 'Mohapatra A', 'Molofsky AB', 'Thornton EE', 'Krummel MF', 'Chawla A', 'Liang HE', 'Locksley RM'], year=2013, doi='10.1038/nature12526')

In [4]:
Abstracts[24037376]

'Eosinophils are specialized myeloid cells associated with allergy and helminth infections. Blood eosinophils demonstrate circadian cycling, as described over 80\u2009years ago, and are abundant in the healthy gastrointestinal tract. Although a cytokine, interleukin (IL)-5, and chemokines such as eotaxins mediate eosinophil development and survival, and tissue recruitment, respectively, the processes underlying the basal regulation of these signals remain unknown. Here we show that serum IL-5 levels are maintained by long-lived type 2 innate lymphoid cells (ILC2) resident in peripheral tissues. ILC2 cells secrete IL-5 constitutively and are induced to co-express IL-13 during type 2 inflammation, resulting in localized eotaxin production and eosinophil accumulation. In the small intestine where eosinophils and eotaxin are constitutive, ILC2 cells co-express IL-5 and IL-13; this co-expression is enhanced after caloric intake. The circadian synchronizer vasoactive intestinal peptide also 

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [5]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [6]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(24037376)
display_summary(24037376, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the abstracts of the papers in our dataset.

We will implement our inverted index as a **Python dictionary with terms as keys and posting lists as values**. For the posting lists, instead of using Python lists and then implementing the different operations on them ourselves, we will use **Python sets** and use the predefined set operations to process these posting "lists". This will also ensure that each document is added at most once per term. The use of Python sets is not the most efficient solution but will work for our purposes. (As an optional additional exercise, you can try to implement the posting lists as Python lists for this and the following assignments.)

Not every paper in our dataset has an abstract; we will only index papers for which an abstract is available.

In [7]:
from collections import defaultdict

inverted_index = defaultdict(set)

# This may take a while:
for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

Let's see what's in the index for the example term 'amsterdam':

In [8]:
print(inverted_index['amsterdam'])

{16513592, 18706961, 15504455}


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'rotterdam':

In [9]:
query_word = 'rotterdam'
for i in inverted_index[query_word]:
    display_summary(i)

----------

# Tasks

**Your name:** Nicholas Kozanidis 2616924

### Task 1

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, such that the function returns a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list. Demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

(You can use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query. You can also exploit the fact that the posting lists are [sets](https://docs.python.org/3/library/stdtypes.html#set), which means you can easily perform set operations such as union, difference and intersect on them.)

In [10]:
def and_query(string):
    newset = set()
    words = tokenize(string)
    wordss = preprocess(words)
    newset = inverted_index[wordss[0]]

    for w in wordss[1:]:
        for s in inverted_index[w]:
            newset &= inverted_index[w]
    
        return newset

for i in and_query('eosinophil signal inflammation'):
    print(i)



2525888
28798252
28863833
10477723
30680604


### Task 2

Construct a second function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query. Demonstrate the working of this second function also with an example (again, choose one that leads to fewer than 100 hits).

In [11]:
def or_query(string):
    newset = set()
    words = tokenize(string)
    wordss = preprocess(words)
    
    for w in wordss:
        for s in inverted_index[w]:
            newset |= inverted_index[w]
    
        return newset

for i in or_query('rotterdam autoimmune'):
    print(i)


14641413
7977294


### Task 3

Show how many hits the query "to be or not to be" returns for your two query functions (`and_query` and `or_query`).

In [14]:
string ='to be or not to be'

for i in or_query(string):
    print(i)
    
for i in and_query(string):
    print(i)



15990785
15990789
15990791
16777227
15990797
25952270
23330844
21233692
6815787
10223679
24641608
18088015
18088022
18088025
8650849
20971618
22282379
23330964
18350229
20971677
17301670
29098153
22020265
31195307
24117436
25166030
30408910
30408913
30408916
16253140
24117462
30408934
24117484
19398907
25952513
8651010
24117513
29098253
8651033
26214693
22806823
3670315
16515372
28573997
17039664
6816051
17039668
6816053
17039670
17039671
17039672
17039673
17039674
17039675
27787580
27263293
24904000
18350400
28836180
16515418
30409058
6816103
28836204
24117618
18612600
20447613
18612606
18612608
19661185
18612613
24117638
18612616
16515467
24117663
7340451
8651174
2359724
9699763
24117686
25428412
9699777
6816199
28049864
25952720
26739157
24117720
28049894
24117737
24117738
27001330
24117773
3146256
17826333
29622825
14680620
14680623
20185660
29622877
24642143
23855711
10224228
787048
2622059
29885036
19923567
31195760
24380018
7864950
23069305
20710010
1573508
657
10224291
3408551


### Task 4

Given the nature of our dataset, how many of the documents from task 3 do you think are actually about the [Shakespeare quote](https://en.wikipedia.org/wiki/To_be%2C_or_not_to_be)? What could you do to better handle such queries? (You don't have to implement this yet!)

Because the quote contains really common words such as articles and prepositions, the results are not in fact relevant to the shakesperian quote.We could use positional index  to get better results for proximity querries or use a combination of biword indexes and positional indexes

### Task 5

Why does `and_query('eosinophil signal inflammation')` not return our example paper 24037376, even though it mentions eosinophil, signal, and inflammation in the abstract? (You do not have to implement anything to fix this yet!)

The abstract contains the words of our query but in forms that are not exact to how we typed them. Because there is no Stemming or Lemmatization process, the function isn't sensitive to plural forms of a word or the roots of it

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.